## Table of Contents

## Imports

In [1]:
from f1_season import F1Season
from prepare_data.lap_data import prepare_lap_data
from prepare_data.weather_data import Weather
from prepare_data.control_message_data import prepare_control_message_data
from prepare_data.driver_data import prepare_driver_data
from prepare_data.race_data import prepare_race_data
import pandas as pd

from fastf1.core import Session

## Execution

In [2]:
season_df = pd.DataFrame()

for i in [2023]:

    end_date = pd.to_datetime('2023-07-07')
    f1_season = F1Season(i, end_date)
    f1_season.update_season_dataframe()

    for j, event in enumerate(f1_season.valid_season_df['SeasonEvents']):
        results = prepare_race_data(event)
        results['LocalOrder'] = j + 1
        for session in event.items():
            if session[0] != 'Race':
                try:
                    prepared_lap_data = prepare_lap_data(session[1])

                    added_weather_data = Weather(prepared_lap_data, session[1])
                    full_dataset = pd.DataFrame()
                    for row in added_weather_data:
                        full_dataset = pd.concat([full_dataset, row])

                    racer_flags = prepare_control_message_data(session[1])
                    updated_full_dataset = pd.merge(full_dataset, racer_flags, on='DriverNumber', how='left') # Move nans to none (str)

                    driver_data = prepare_driver_data(session[1])
                    # You can merge this for each session in event
                    updated_full_dataset = pd.merge(updated_full_dataset, driver_data, on='DriverNumber', how='left')
                    updated_full_dataset = pd.merge(updated_full_dataset, results, on='DriverNumber', how='inner')

                    updated_full_dataset['SessionType'] = session[0]
                    updated_full_dataset['SeasonYear'] = i 
                    
                    season_df = pd.concat([season_df, updated_full_dataset], ignore_index=True)
                except:
                    print(f"Session: {session[1]}")


In [12]:
updated_full_dataset['Category'].unique()

array([nan, 'Flag', 'CarEvent'], dtype=object)

In [6]:
season_df

,Driver,DriverNumber,Time_min,Time_max,LapTimeSeconds_min,LapTimeSeconds_max,LapTimeSeconds_mean,LapTimeSeconds_std,LapTimeSeconds_count,Sector1TimeSeconds_min,Sector1TimeSeconds_max,Sector1TimeSeconds_mean,Sector1TimeSeconds_std,Sector2TimeSeconds_min,Sector2TimeSeconds_max,Sector2TimeSeconds_mean,Sector2TimeSeconds_std,Sector3TimeSeconds_min,Sector3TimeSeconds_max,Sector3TimeSeconds_mean,Sector3TimeSeconds_std,SpeedI1_min,SpeedI1_max,SpeedI1_mean,SpeedI1_std,SpeedI2_min,SpeedI2_max,SpeedI2_mean,SpeedI2_std,SpeedFL_min,SpeedFL_max,SpeedFL_mean,SpeedFL_std,SpeedST_min,SpeedST_max,SpeedST_mean,SpeedST_std,IsPersonalBest_pr_lap,AirTemp_min,AirTemp_max,AirTemp_mean,AirTemp_std,Humidity_min,Humidity_max,Humidity_mean,Humidity_std,Pressure_min,Pressure_max,Pressure_mean,Pressure_std,TrackTemp_min,TrackTemp_max,TrackTemp_mean,TrackTemp_std,WindDirection_min,WindDirection_max,WindDirection_mean,WindDirection_std,WindSpeed_min,WindSpeed_max,WindSpeed_mean,WindSpeed_std,TeamId,CountryCode,Position,Points,Country,Location,EventName,LocalOrder,SessionType,SeasonYear
0,ALB,23,0 days 00:29:46.931000,0 days 01:18:00.966000,96.018,137.05,108.091714,19.343659,7,30.341,42.034,33.287429,4.665395,41.571,63.859,46.671714,8.863485,23.934,43.986,28.132571,7.459461,115.0,236.0,204.857143,52.454515,187.0,270.0,246.0,36.175498,64.0,287.0,250.142857,82.136936,253.0,321.0,296.428571,29.004926,5,26.5,27.3,26.850000,0.241450,11.0,14.0,12.500000,0.652328,1017.2,1017.5,1017.308333,0.106857,38.4,41.1,39.920833,0.802379,0.0,359.0,287.312500,127.497377,1.7,3.5,2.429167,0.484201,williams,THA,10.0,1.0,Bahrain,Sakhir,Bahrain Grand Prix,1,Practice 1,2023
1,ALO,14,0 days 00:19:50.339000,0 days 01:18:37.672000,93.196,140.868,112.108071,18.372001,14,29.654,43.517,34.973,5.303451,40.156,65.8,49.373071,9.643082,23.386,40.274,27.762,5.144361,87.0,240.0,192.285714,52.19848,97.0,266.0,207.428571,63.752785,48.0,283.0,254.357143,67.160665,243.0,312.0,291.307692,24.01522,11,26.5,27.4,26.913559,0.276965,11.0,14.0,12.406780,0.646440,1017.2,1017.5,1017.310169,0.107791,38.4,41.6,40.128814,0.921590,0.0,359.0,292.542373,122.761518,1.7,3.5,2.447458,0.458391,aston_martin,ESP,3.0,15.0,Bahrain,Sakhir,Bahrain Grand Prix,1,Practice 1,2023
2,BOT,77,0 days 00:28:49.439000,0 days 01:17:51.965000,94.689,134.895,103.471444,13.923024,9,30.14,37.482,32.318889,2.65654,40.78,56.747,44.428667,5.72371,23.769,41.944,26.723889,5.993527,181.0,236.0,222.111111,19.57961,153.0,263.0,242.222222,37.198044,64.0,278.0,248.555556,69.318308,242.0,305.0,280.333333,23.895606,6,26.5,27.3,26.853061,0.239880,11.0,14.0,12.489796,0.649437,1017.2,1017.5,1017.306122,0.106865,38.4,41.1,39.944898,0.811650,0.0,359.0,281.653061,132.235987,1.7,3.5,2.424490,0.480248,alfa,FIN,8.0,4.0,Bahrain,Sakhir,Bahrain Grand Prix,1,Practice 1,2023
3,DEV,21,0 days 00:19:18.449000,0 days 01:17:19.877000,95.402,143.967,112.668312,17.814844,16,30.314,43.749,34.242938,5.078408,41.419,65.276,49.654187,8.719672,23.669,43.15,28.771187,5.952843,147.0,236.0,212.3125,33.863882,137.0,265.0,218.75,44.22895,54.0,279.0,253.3125,56.402977,230.0,312.0,288.866667,24.514913,8,26.5,27.4,26.934483,0.272449,11.0,14.0,12.362069,0.667498,1017.2,1017.5,1017.303448,0.105905,38.5,41.7,40.215517,0.891638,0.0,359.0,291.741379,123.676369,1.7,3.5,2.448276,0.462351,alphatauri,NED,14.0,0.0,Bahrain,Sakhir,Bahrain Grand Prix,1,Practice 1,2023
4,GAS,10,0 days 00:19:41.105000,0 days 01:17:40.471000,95.455,147.464,113.915533,22.793223,15,30.404,51.783,37.392067,9.077354,41.2,68.723,49.829867,10.919426,23.833,41.265,26.6936,4.926008,89.0,233.0,187.266667,58.817474,113.0,262.0,218.533333,58.461057,63.0,282.0,264.8,55.845706,263.0,313.0,297.4,18.500193,11,26.5,27.4,26.920690,0.273867,11.0,14.0,12.396552,0.647252,1017.2,1017.5,1017.306897,0.105734,38.4,41.6,40.158621,0.900494,0.0,359.0,291.758621,123.684702,1.7,3.5,2.446552,0.462342,alpine,FRA,9.0,2.0,Bahrain,Sakhir,Bahrain Grand Prix,1,Practice 1,2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

## Testing Catboost

In [14]:
from catboost import CatBoostRegressor, Pool

In [15]:
season_df = season_df.fillna(-1.0)

In [16]:
model_data = season_df.drop(['Time_min', 'Time_max'], axis=1)
model_data[['DriverNumber', 'LocalOrder']] = model_data[['DriverNumber', 'LocalOrder']].astype(object)

In [21]:
train_data = model_data.query('SeasonYear < 2023 or LocalOrder < 3')
val_data = model_data.query('LocalOrder in (3, 4) & SeasonYear == 2023')
test_data = model_data.query('LocalOrder >= 5 & SeasonYear == 2023')

In [23]:
X_train = train_data.drop(['Points', 'Position'], axis=1)
y_train = train_data.Points
X_val = val_data.drop(['Points', 'Position'], axis=1)
y_val = val_data.Points

X_test = test_data.drop(['Points', 'Position'], axis=1)
y_test = test_data.Points

In [24]:
categorical_columns_indices = [i for i, dtype in enumerate(X_train.dtypes) if dtype == object]

In [25]:
train_pool = Pool(X_train, y_train, cat_features=categorical_columns_indices)
val_pool = Pool(X_val, y_val, cat_features=categorical_columns_indices)

In [ ]:
ray.shutdown()
ray.init(num_gpus=1)

def objective(config):

    # Set the CatBoostRegressor parameters based on the config
    model = CatBoostClassifier(
        learning_rate=config["learning_rate"],
        depth=int(config["depth"]),
        l2_leaf_reg=int(config["l2_leaf_reg"]),
        min_data_in_leaf=int(config["min_data_in_leaf"]),
        iterations=100,
        thread_count=1,
        gpu_ram_part=0.2,
        task_type='GPU'
    )

    # Train the model
    model.fit(X_train, 
              y_train, 
              eval_set=(X_val, y_val), 
              verbose=False, 
              early_stopping_rounds=2, 
              cat_features=categorical_features_indices)

    # Evaluate the model
    score = model.get_evals_result()

    loss = score['validation']['Logloss'][-1]
    session.report({"log_loss": loss, "done": True})
    # return loss

space = {
    "learning_rate": tune.loguniform(1e-3, 1),
    "depth": tune.randint(4, 12),
    "l2_leaf_reg": tune.randint(1, 10),
    "min_data_in_leaf": tune.randint(1, 40)
}

hyperopt_search = HyperOptSearch(metric="log_loss", mode="min")
trainable_with_cpu_gpu = tune.with_resources(objective, {"cpu" : 1, "gpu": 0.2})

tuner = tune.Tuner(
    trainable_with_cpu_gpu,
    tune_config=tune.TuneConfig(max_concurrent_trials=4, num_samples=10),
    param_space=space,
)
results = tuner.fit()

In [27]:
model = CatBoostRegressor(eval_metric='RMSE')
model.fit(train_pool, eval_set=val_pool, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.068532
0:	learn: 6.4186662	test: 7.1400738	best: 7.1400738 (0)	total: 7.16ms	remaining: 7.15s
1:	learn: 6.2053332	test: 6.9872880	best: 6.9872880 (1)	total: 14.1ms	remaining: 7.02s
2:	learn: 6.0107104	test: 6.8378140	best: 6.8378140 (2)	total: 19.4ms	remaining: 6.44s
3:	learn: 5.8354732	test: 6.7027155	best: 6.7027155 (3)	total: 24.6ms	remaining: 6.13s
4:	learn: 5.6803269	test: 6.5920843	best: 6.5920843 (4)	total: 29.5ms	remaining: 5.87s
5:	learn: 5.5427684	test: 6.4881284	best: 6.4881284 (5)	total: 34.4ms	remaining: 5.7s
6:	learn: 5.4095835	test: 6.3999824	best: 6.3999824 (6)	total: 38.4ms	remaining: 5.45s
7:	learn: 5.2843846	test: 6.3329619	best: 6.3329619 (7)	total: 42.7ms	remaining: 5.29s
8:	learn: 5.1839904	test: 6.2868908	best: 6.2868908 (8)	total: 45.9ms	remaining: 5.05s
9:	learn: 5.0709562	test: 6.2553322	best: 6.2553322 (9)	total: 50.9ms	remaining: 5.04s
10:	learn: 4.9805825	test: 6.2141101	best: 6.2141101 (10)	total: 54.1ms	remaining: 4.87s
11:	learn: 4

In [28]:
feature_importance = pd.DataFrame({'Feature': model.feature_names_, 'Importance': model.feature_importances_})

In [29]:
feature_importance.sort_values('Importance', ascending=False).reset_index(drop=True).iloc[0:20]

,Feature,Importance
0,TeamId,38.727030
1,DriverNumber,9.270728
2,Driver,6.217101
3,SeasonYear,4.471120
4,LocalOrder,3.609974
5,EventName,3.428820
6,CountryCode,2.064586
7,Country,1.477094
8,Location,1.454387
9,LapTimeSeconds_min,1.426726


In [30]:
predictions = model.predict(X_test)

In [31]:
results = X_test[['Driver', 'DriverNumber', 'TeamId', 'EventName', 'LocalOrder', 'SessionType']].copy()
results['Predictions'] = predictions
results['Actuals'] = y_test
results['Position'] = test_data.Position

In [32]:
results_first_race = results.query('LocalOrder == 5')
results_second_race = results.query('LocalOrder == 6')
results_third_race = results.query('LocalOrder == 7')

In [33]:
test_results_first = pd.DataFrame(results_first_race.groupby(['Driver', 'DriverNumber', 'EventName', 'Position', 'Actuals'])['Predictions'].mean())
test_results_second = pd.DataFrame(results_second_race.groupby(['Driver', 'DriverNumber', 'EventName', 'Position','Actuals'])['Predictions'].mean())
test_results_third = pd.DataFrame(results_third_race.groupby(['Driver', 'DriverNumber', 'EventName', 'Position','Actuals'])['Predictions'].mean())

In [42]:
test_results_third.sort_values('Predictions', ascending=False)

,,,,,Predictions
Driver,DriverNumber,EventName,Position,Actuals,
VER,1,Canadian Grand Prix,1.0,25.0,16.465108
HAM,44,Canadian Grand Prix,3.0,15.0,11.690102
LEC,16,Canadian Grand Prix,4.0,12.0,11.019449
PER,11,Canadian Grand Prix,6.0,9.0,9.404466
SAI,55,Canadian Grand Prix,5.0,10.0,8.104420
ALO,14,Canadian Grand Prix,2.0,18.0,6.102076
RUS,63,Canadian Grand Prix,19.0,0.0,5.856735
OCO,31,Canadian Grand Prix,8.0,4.0,4.949282
GAS,10,Canadian Grand Prix,12.0,0.0,4.761311


## Procedure

1) Create Season from **F1Season()** class and update dataframe from **update_season_dataframe()** method
2) For each session, pass lap data into **prepare_lap_data()** function
3) For each row in this lap data, pass into **weather_for_racer()** function
4) Join control message data to session data
5) Join driver data to session data

## Things to do/fix

1) Case when a driver has multiple control messsages in a session
2) Add position and points as potential response variables
3) Add relative driver and team points at time of pre-race (or wait till all data is pulled)